### **LIBRERÍAS**

In [2]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
#from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt
import a_funciones as fn

### **CARGAR LOS DATOS**

In [3]:
conn=sql.connect('data\\db_movies') ### crear cuando no existe el nombre de cd  y para conectarse cuando sí existe.
cur=conn.cursor() ###para funciones que ejecutan sql en base de datos

In [4]:
cur.execute('select name from sqlite_master where type = "table"')
cur.fetchall()

[('ratings',),
 ('movies',),
 ('view_mov',),
 ('merge_ratings',),
 ('user_sel',),
 ('movies_sel',),
 ('ratings_filtered',),
 ('movies_final',),
 ('ratings_final',)]

**Nombrar las tablas**

In [5]:
movies = pd.read_sql("SELECT * from movies", conn)
ratings = pd.read_sql("SELECT * from ratings", conn)

In [6]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


### VERIFICAR TÍTULOS DUPLICADOS

In [7]:
dup_titles = pd.read_sql("""SELECT title, COUNT(*) as count
                         FROM movies
                         GROUP BY title
                         HAVING COUNT(*) > 1;""", conn)

dup_titles

,title,count
0,Confessions of a Dangerous Mind (2002),2
1,Emma (1996),2
2,Eros (2004),2
3,Saturn 3 (1980),2
4,War of the Worlds (2005),2


Se tiene una primera tabla *movies*, la cual contiene información sobre las películas (título, año de lanzamineto y los géneros asociados a la misma)

In [ ]:
ratings

Se tiene también, una segunda tabla, llamada *ratings*, la cual contiene información asociada a las calificaciones que ha obtenido una película y la cantidad de usuarios que han calificado la película.

**Separar el año en una nueva columna**

In [8]:
query = """
SELECT 
    movieId, 
    TRIM(SUBSTR(title, 1, INSTR(title, '(') - 1)) AS title,  -- Elimina el año del título
    SUBSTR(title, INSTR(title, '(') + 1, 4) AS year,         -- Extrae el año
    genres
FROM 
    movies;
"""

movies2 = pd.read_sql(query, conn)

#DEJAR PARA DESPUÉS DEL TRATAMIENTO

In [9]:
movies2

,movieId,title,year,genres
0,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,1995,Adventure|Children|Fantasy
2,3,Grumpier Old Men,1995,Comedy|Romance
3,4,Waiting to Exhale,1995,Comedy|Drama|Romance
4,5,Father of the Bride Part II,1995,Comedy
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,2017,Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero,2017,Animation|Comedy|Fantasy
9739,193585,Flint,2017,Drama
9740,193587,Bungo Stray Dogs: Dead Apple,2018,Action|Animation


In [10]:
movies2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   year     9742 non-null   object
 3   genres   9742 non-null   object
dtypes: int64(1), object(3)
memory usage: 304.6+ KB


In [ ]:
'''
genres=movies["genres"].str.split("|")
te = TransactionEncoder()
genres = te.fit_transform(genres)
genres = pd.DataFrame(genres, columns = te.columns_)
len(movies["genres"].unique())
'''

**Dummizar la columna de género, separando los carácteres contenidos en la misma.** 
Esta dummización se hace con el fin de poder analizar más fácilmente la base de datos

In [11]:
# Separar los géneros en columnas teniendo en cuenta el criterio de separación '|'
genres_dummies = movies2['genres'].str.get_dummies(sep='|')

# Concatenar las columnas de géneros con el DataFrame original
movies3 = pd.concat([movies2, genres_dummies], axis=1)

In [12]:
movies3

,movieId,title,year,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,Adventure|Children|Fantasy,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,1995,Comedy|Romance,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,1995,Comedy|Drama|Romance,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II,1995,Comedy,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,2017,Action|Animation|Comedy|Fantasy,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9738,193583,No Game No Life: Zero,2017,Animation|Comedy|Fantasy,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9739,193585,Flint,2017,Drama,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9740,193587,Bungo Stray Dogs: Dead Apple,2018,Action|Animation,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
movies3 = movies3.rename(columns={"(no genres listed)": "no_genre"})
movies3

,movieId,title,year,genres,no_genre,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,Adventure|Children|Fantasy,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,1995,Comedy|Romance,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,1995,Comedy|Drama|Romance,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II,1995,Comedy,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,2017,Action|Animation|Comedy|Fantasy,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9738,193583,No Game No Life: Zero,2017,Animation|Comedy|Fantasy,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9739,193585,Flint,2017,Drama,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9740,193587,Bungo Stray Dogs: Dead Apple,2018,Action|Animation,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
no_genre = movies3[movies3['no_genre']==1]
no_genre

,movieId,title,year,genres,no_genre,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
8517,114335,La cravate,1957,(no genres listed),1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8684,122888,Ben-hur,2016,(no genres listed),1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8687,122896,Pirates of the Caribbean: Dead Men Tell No Tales,2017,(no genres listed),1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8782,129250,Superfast!,2015,(no genres listed),1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8836,132084,Let It Be Me,1995,(no genres listed),1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8902,134861,Trevor Noah: African American,2013,(no genres listed),1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9033,141131,Guardians,2016,(no genres listed),1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9053,141866,Green Room,2015,(no genres listed),1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9070,142456,The Brand New Testament,2015,(no genres listed),1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9091,143410,,Hyen,(no genres listed),1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
genre_totals = movies3.iloc[:, 4:].sum().sort_values(ascending=False)
genre_totals

Drama          4361
Comedy         3756
Thriller       1894
Action         1828
Romance        1596
Adventure      1263
Crime          1199
Sci-Fi          980
Horror          978
Fantasy         779
Children        664
Animation       611
Mystery         573
Documentary     440
War             382
Musical         334
Western         167
IMAX            158
Film-Noir        87
no_genre         34
dtype: int64

In [16]:
genre_totals_df = genre_totals.reset_index()
genre_totals_df.columns = ['Genre', 'Count']

In [17]:
# Crear el gráfico de barras
fig = go.Figure()

# Agregar datos al gráfico
fig.add_trace(go.Bar(
    x=genre_totals_df['Genre'],
    y=genre_totals_df['Count'],
    marker_color='indigo'  # Puedes cambiar el color si deseas
))

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Total de Géneros de Películas',
    xaxis_title='Género',
    yaxis_title='Cantidad',
    xaxis_tickangle=-45,
    template='plotly_white'  # Plantilla del gráfico
)

# Mostrar el gráfico
fig.show()

### **VISUALIZACIÓN DE LOS DATOS**

**Calificaciones generales**

In [ ]:
cr = pd.read_sql("""
    SELECT 
        "rating" AS rating,
        COUNT(*) AS conteo,
        (COUNT(*) * 100.0 / SUM(COUNT(*)) OVER ()) AS porcentaje
    FROM ratings
    GROUP BY "rating"
    ORDER BY "rating"
""", conn)

cr

In [ ]:
pd.read_sql("select count(*) from ratings", conn)

# Definir los colores según las calificaciones
colors = []
for rating in cr['rating']:
    if rating == 0.5:
        colors.append('#264653')  
    elif 1 <= rating <= 2:
        colors.append('#fe4a49')  
    elif 2.5 <= rating <= 3.5:
        colors.append('#fed766')  
    elif 4 <= rating <= 5:
        colors.append('#009fb7')  

data  = go.Bar( x=cr.rating,y=cr.conteo, text=cr.conteo, textposition="outside", marker_color=colors)

layout = go.Layout(
    title={
        'text': "Conteo de Calificaciones",
        'y': 0.94,
        'x': 0.5, 
        'xanchor': 'center',  # Anclar el título al centro
        'yanchor': 'top'
    },
    xaxis={
        'title': 'Calificación',
        'tickvals': cr['rating']  # Asegurar que todos los valores del eje X se muestren
    }, 
    yaxis={'title': 'Cantidad'},
    width=800,   # Ancho del gráfico
    height=600   # Alto del gráfico
)

# Crear la figura y mostrar
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
ratings

**Calificaciones por usuario**

In [ ]:
rating_users=pd.read_sql(''' SELECT "userId" as user_id,
                         count(*) as cnt_rat
                         FROM ratings
                         group by "userId"
                         order by cnt_rat asc
                         ''',conn)

rating_users

In [ ]:
fn.plot_histogram(rating_users, 'cnt_rat', bins=20, color='#264653')

*Según un nuevo estudio, una persona promedio pasa más de 78.000 horas frente al televisor a lo largo de su vida. Los investigadores determinaron que los espectadores ven un promedio de 3.639 películas y 31.507 episodios de televisión, lo que equivale a unas asombrosas 78.705 horas de televisión por hora.*

Filtro para ver los usuarios que han visto más de X película|s y menos de 1000

In [ ]:
rating_users2=pd.read_sql(''' select "userId" as user_id,
                         count(*) as cnt_rat
                         FROM ratings
                         group by "userId"
                         having cnt_rat <=1000
                         order by cnt_rat asc
                         ''',conn )

rating_users2

In [ ]:
fn.plot_histogram(rating_users2, 'cnt_rat', bins=20, color='#264653')

In [ ]:
#### verificar cuantas calificaciones tiene cada película
rating_movies=pd.read_sql(''' select movieId ,
                         count(*) as cnt_rat
                         from ratings
                         group by "movieId"
                         order by cnt_rat desc
                         ''',conn )

rating_movies

In [ ]:
ratings

In [ ]:
fn.plot_histogram(rating_movies, 'cnt_rat', bins=20, color='#264653')

In [ ]:
#### verificar cuantas calificaciones tiene cada película
rating_movies2=pd.read_sql(''' select movieId ,
                         count(*) as cnt_rat
                         from ratings
                         group by "movieId"
                         having cnt_rat >= 20
                         order by cnt_rat desc
                         ''',conn )

rating_movies2

In [ ]:
fn.plot_histogram(rating_movies2, 'cnt_rat', bins=20, color='#264653')

In [73]:
ratings_list = np.arange(0.5, 5.5, 0.5)

# Contar cuántas veces calificó cada usuario
rating_counts = ratings.groupby('userId')['rating'].value_counts().unstack(fill_value=0)

# Reindexar para asegurarse de que todas las calificaciones estén presentes
rating_counts = rating_counts.reindex(columns=ratings_list, fill_value=0)

# Añadir la columna de total de calificaciones
rating_counts['Total'] = rating_counts.sum(axis=1)

rating_counts = rating_counts.sort_values(by='Total', ascending=False)

In [ ]:
rating_counts

In [ ]:
# Crear la nueva matriz binaria
binary_matrix = (rating_counts.iloc[:, :-1] >= 1).astype(int)

# Calcular la columna Total en la matriz binaria
binary_matrix['Total'] = binary_matrix.sum(axis=1)

# Mostrar la nueva matriz con Total
binary_matrix

In [76]:
id_binary = binary_matrix.index.tolist()

In [77]:
binary_total = binary_matrix['Total'].tolist()

In [ ]:
# Unión de ID con Total
df_binary = pd.DataFrame()
df_binary['userId'] = id_binary
df_binary['Total_ranges'] = binary_total
df_binary

In [ ]:
# Pegar total de 'df_binary' en 'rating_counts' según 'userId'
df_merged_ratings = pd.merge(rating_counts,df_binary, on=['userId'], how='outer')
df_merged_ratings.sort_values(by='Total', ascending=False)

In [ ]:
# Contar cuántos usuarios tienen cada total
total_counts = binary_matrix['Total'].value_counts().sort_index()

# Graficar
fig = go.Figure()

fig.add_trace(go.Bar(
    x=total_counts.index,  # Total de calificaciones
    y=total_counts.values,  # Número de usuarios
    marker=dict(color='#264653')
))

# Personalizar el diseño del gráfico
fig.update_layout(
    title='Cantidad de Usuarios por Total de Calificaciones',
    xaxis_title='Total de Calificaciones',
    yaxis_title='Número de Usuarios',
    width=800,
    height=600,
)

# Mostrar el gráfico
fig.show()

In [ ]:
# Definir los bins manualmente
bins = [0, 2, 4, 6, 11]  # Bins para los rangos (1-2), (3-4), (5-6), (+6)
labels = ['1-2', '3-4', '5-6', '+6']  # Etiquetas para los bins

# Agrupar los totales en los bins definidos
total_binned = pd.cut(total_counts.index, bins=bins, labels=labels, right=True)

# Contar cuántos usuarios caen en cada bin
binned_counts = total_counts.groupby(total_binned).sum().reindex(labels, fill_value=0)

# Graficar
fig = go.Figure()

fig.add_trace(go.Bar(
    x=binned_counts.index,  # Total de calificaciones en bins
    y=binned_counts.values,  # Número de usuarios en cada bin
    marker=dict(color='#264653')
))

# Personalizar el diseño del gráfico
fig.update_layout(
    title='Cantidad de Usuarios por Total de Calificaciones (Binned)',
    xaxis_title='Total de Calificaciones (bins)',
    yaxis_title='Número de Usuarios',
    width=800,
    height=600,
)

# Mostrar el gráfico
fig.show()

In [ ]:
ratings

## **Preprocesamiento .PY**

In [5]:
fn.ejecutar_sql('preprocesamiento.sql', cur)

In [ ]:
cur.execute("select name from sqlite_master where type='table' ")
cur.fetchall()

In [ ]:
movies

In [ ]:
pd.read_sql('select count(*) movies_final', conn)
movies_final =pd.read_sql('select * from  movies_final',conn)
movies_final

In [ ]:
ratings

In [ ]:
pd.read_sql('select count(*) ratings_filtered', conn)
ratings_filtered =pd.read_sql('select * from  ratings_filtered',conn)
ratings_filtered

In [ ]:
pd.read_sql('select count(*) ratings_final', conn)
df_final = pd.read_sql('select * from  ratings_final',conn)
df_final

In [11]:
df_final.drop('movie_id', axis=1, inplace=True)
df_final.drop('movieId:1', axis=1, inplace=True)

In [12]:
query = """SELECT 
    TRIM(SUBSTRING(title, 1, LENGTH(title) - 7)) AS movie_name,  -- Extrae el nombre de la película
    SUBSTRING(title, LENGTH(title) - 4, 4) AS movie_year          -- Extrae el año
FROM movies;"""

df_final = pd.read_sql(query, conn)

In [ ]:
df_final.head()

In [ ]:
df_final